In [1]:
load("../Sage/SymbolicVectorMatrix.sage")

In [2]:
num_stages = 3
# shu osher form
a_s = get_matrix_variable("a_s", num_stages + 1, num_stages + 1)
b_s = get_matrix_variable("b_s", num_stages + 1, num_stages + 1)
c_s = get_vector_variable("c_s", num_stages + 1)
c_s[0] = 0
c_s[num_stages] = 1
for i in range(num_stages + 1):
    a_s[i, i:] = 0
    b_s[i, i:] = 0
    
# butcher form
a_b = get_matrix_variable("a_b", num_stages, num_stages)
b_b = get_vector_variable("b_b", num_stages)
c_b = get_vector_variable("c_b", num_stages)
c_b[0] = 0
for i in range(num_stages):
    a_b[i, i:] = 0

In [51]:
q_n = var("q_n")
t_n = var("t_n")
t = var("t")
q = var("q")
delta_t = var("delta_t")
Lfunc = function("L", nargs=2)(t, q)


# butcher form
k = get_vector_variable("k", num_stages)
k_dict_ = dict()
for i in range(num_stages):
    k_dict_[k[i]] = Lfunc(t=t_n + c_b[i] * delta_t, q=q_n + delta_t * sum([a_b[i, j] * k[j] for j in range(0, i)]))
q_np1_b = q_n + delta_t * sum([b_b[i] * k[i] for i in range(num_stages)])

# shu_osher_form
y = get_vector_variable("y", num_stages + 1)
L = get_vector_variable("L", num_stages)
y_dict_ = dict()
y_dict_[y[0]] = q_n
for i in range(1, num_stages+1):
    y_dict_[y[i]] = sum([a_s[i,j] * y[j] + b_s[i,j] * delta_t * L[j] for j in range(0,i)])
    
L_dict_ = dict()
for i in range(num_stages):
    L_dict_[L[i]] = Lfunc(t=t_n + c_s[i] * delta_t, q=y[i])
q_np1_s = y[num_stages]

def shu_osher_form(a_s, b_s, c_s):
    num_stages = a_s.nrows() - 1
    y = vector(SR, num_stages + 1)
    L = vector(SR, num_stages)
    y[0] = q_n
    for i in range(1, num_stages+1):
        y[i] = sum([a_s[i,j] * y[j] + b_s[i,j] * delta_t * Lfunc(t=t_n + c_s[j] * delta_t, q=y[j]) for j in range(i)])

    q_np1_s = y[num_stages]
    return q_np1_s

def butcher_form(a_b, b_b, c_b):
    num_stages = a_b.nrows()
    k = vector(SR, num_stages)
    
    for i in range(num_stages):
        k[i] = Lfunc(t=t_n + c_b[i] * delta_t, q=q_n + delta_t * sum([a_b[i, j] * k[j] for j in range(0, i)]))
    q_np1_b = q_n + delta_t * sum([b_b[i] * k[i] for i in range(num_stages)])
    return q_np1_b

In [4]:
def convert_shu_osher_to_butcher_form(a_s, b_s, c_s):
    num_stages = a_s.nrows() - 1
    c_b = c_s[0:-1]
    
    m = sum([a_s ** j * b_s for j in range(num_stages)])
    a_b = m[:num_stages, :num_stages]
    b_b = vector(m[-1, :num_stages])
    return (a_b, b_b, c_b)

In [23]:
def convert_butcher_to_shu_osher_form(a_b, b_b, c_b):
    # This is not a unique transformation
    num_stages = a_b.nrows()
    list_ = c_b.list()
    list_.append(1)
    c_s = vector(list_)
    
    a_s = matrix(ring=SR, nrows=num_stages + 1, ncols=num_stages + 1)
    b_s = matrix(ring=SR, nrows=num_stages + 1, ncols=num_stages + 1)
    
    a_s[1:, 0] = 1
    b_s[:num_stages, :num_stages] = a_b[:, :]
    b_s[num_stages, :num_stages] = b_b[:]
    
    return (a_s, b_s, c_s)

In [24]:
tuple_ = convert_butcher_to_shu_osher_form(a_b, b_b, c_b)
print(tuple_[0])
print(tuple_[1])
print(tuple_[2])

[0 0 0 0]
[1 0 0 0]
[1 0 0 0]
[1 0 0 0]
[      0       0       0       0]
[a_b_1_0       0       0       0]
[a_b_2_0 a_b_2_1       0       0]
[  b_b_0   b_b_1   b_b_2       0]
(0, c_b_1, c_b_2, 1)


In [64]:
tuple_ = convert_shu_osher_to_butcher_form(a_s, b_s, c_s)
print(tuple_[0])
print(tuple_[1])
print(tuple_[2])

[                        0                         0                         0]
[                  b_s_1_0                         0                         0]
[a_s_2_1*b_s_1_0 + b_s_2_0                   b_s_2_1                         0]
(a_s_2_1*a_s_3_2*b_s_1_0 + a_s_3_1*b_s_1_0 + a_s_3_2*b_s_2_0 + b_s_3_0, a_s_3_2*b_s_2_1 + b_s_3_1, b_s_3_2)
(0, c_s_1, c_s_2)


In [52]:
shu_osher_form(tuple_[0], tuple_[1], tuple_[2])

b_b_1*delta_t*L(c_b_1*delta_t + t_n, a_b_1_0*delta_t*L(t_n, q_n) + q_n) + b_b_2*delta_t*L(c_b_2*delta_t + t_n, a_b_2_1*delta_t*L(c_b_1*delta_t + t_n, a_b_1_0*delta_t*L(t_n, q_n) + q_n) + a_b_2_0*delta_t*L(t_n, q_n) + q_n) + b_b_0*delta_t*L(t_n, q_n) + q_n

In [46]:
butcher_form(a_b, b_b, c_b)

(b_b_1*L(c_b_1*delta_t + t_n, a_b_1_0*delta_t*L(t_n, q_n) + q_n) + b_b_2*L(c_b_2*delta_t + t_n, (a_b_2_1*L(c_b_1*delta_t + t_n, a_b_1_0*delta_t*L(t_n, q_n) + q_n) + a_b_2_0*L(t_n, q_n))*delta_t + q_n) + b_b_0*L(t_n, q_n))*delta_t + q_n

In [55]:
tuple_ = convert_butcher_to_shu_osher_form(a_b, b_b, c_b)
(butcher_form(a_b, b_b, c_b) - shu_osher_form(tuple_[0], tuple_[1], tuple_[2])).full_simplify()

0

In [61]:
tuple_ = convert_shu_osher_to_butcher_form(a_s, b_s, c_s)
(butcher_form(tuple_[0], tuple_[1], tuple_[2]) - shu_osher_form(a_s, b_s, c_s)).full_simplify()

-(a_s_3_2*b_s_2_1 + b_s_3_1)*delta_t*L(c_s_1*delta_t + t_n, b_s_1_0*delta_t*L(t_n, q_n) + a_s_1_0*q_n) + (a_s_3_2*b_s_2_1 + b_s_3_1)*delta_t*L(c_s_1*delta_t + t_n, b_s_1_0*delta_t*L(t_n, q_n) + q_n) - b_s_3_2*delta_t*L(c_s_2*delta_t + t_n, b_s_2_1*delta_t*L(c_s_1*delta_t + t_n, b_s_1_0*delta_t*L(t_n, q_n) + a_s_1_0*q_n) + (a_s_2_1*b_s_1_0 + b_s_2_0)*delta_t*L(t_n, q_n) + (a_s_1_0*a_s_2_1 + a_s_2_0)*q_n) + b_s_3_2*delta_t*L(c_s_2*delta_t + t_n, b_s_2_1*delta_t*L(c_s_1*delta_t + t_n, b_s_1_0*delta_t*L(t_n, q_n) + q_n) + (a_s_2_1*b_s_1_0 + b_s_2_0)*delta_t*L(t_n, q_n) + q_n) - (a_s_1_0*a_s_3_1 + (a_s_1_0*a_s_2_1 + a_s_2_0)*a_s_3_2 + a_s_3_0 - 1)*q_n

In [123]:
a_s_fe = matrix([[0, 0], [1, 0]])
b_s_fe = matrix([[0, 0], [1, 0]])
c_s_fe = vector([0, 1])
a_b_fe = matrix([[0]])
b_b_fe = vector([1])
c_b_fe = vector([0])

tuple_ = convert_shu_osher_to_butcher_form(a_s_fe, b_s_fe, c_s_fe)
assert tuple_[0] == a_b_fe
assert tuple_[1] == b_b_fe
assert tuple_[2] == c_b_fe

tuple_ = convert_butcher_to_shu_osher(a_b_fe, b_b_fe, c_b_fe)
assert tuple_[0] == a_s_fe
assert tuple_[1] == b_s_fe
assert tuple_[2] == c_s_fe

In [63]:
a_s_tvdrk2 = matrix([[0, 0, 0], [1, 0, 0], [1/2, 1/2, 0]])
b_s_tvdrk2 = matrix([[0, 0, 0], [1, 0, 0], [0, 1/2, 0]])
c_s_tvdrk2 = vector([0, 1, 1])
a_b_tvdrk2 = matrix([[0, 0], [1, 0]])
b_b_tvdrk2 = vector([1/2, 1/2])
c_b_tvdrk2 = vector([0, 1])

tuple_ = convert_shu_osher_to_butcher_form(a_s_tvdrk2, b_s_tvdrk2, c_s_tvdrk2)
assert tuple_[0] == a_b_tvdrk2
assert tuple_[1] == b_b_tvdrk2
assert tuple_[2] == c_b_tvdrk2

tuple_ = convert_butcher_to_shu_osher_form(a_b_tvdrk2, b_b_tvdrk2, c_b_tvdrk2)
assert tuple_[0] == a_s_tvdrk2
assert tuple_[1] == b_s_tvdrk2
assert tuple_[2] == c_s_tvdrk2

AssertionError: 